In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import pandas as pd

In [17]:
dataset = pd.read_json("Sarcasm_Headlines_Dataset.json", lines=True)
dataset.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [18]:
column_name_to_remove = 'article_link'
dataset = dataset.drop(columns=[column_name_to_remove])

dataset.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [19]:
import pandas as pd

# Read the dataset
dataset = pd.read_json("Sarcasm_Headlines_Dataset.json", lines=True)

# Get the number of rows in the dataset
num_rows = dataset.shape[0]

# Display the total number of rows
print("Total number of rows in the dataset:", num_rows)


Total number of rows in the dataset: 26709


In [20]:
import pandas as pd

# Read the dataset
dataset = pd.read_json("Sarcasm_Headlines_Dataset.json", lines=True)

# Assuming "is_sarcastic" is the column you're interested in
sarcastic_counts = dataset['is_sarcastic'].value_counts()

# Display the result
print("Number of rows for each value in the 'is_sarcastic' column:")
print(sarcastic_counts)



Number of rows for each value in the 'is_sarcastic' column:
is_sarcastic
0    14985
1    11724
Name: count, dtype: int64


In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

# Read the dataset
dataset = pd.read_json("Sarcasm_Headlines_Dataset.json", lines=True)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dataset['headline'], dataset['is_sarcastic'], test_size=0.2, random_state=42)

# Tokenize and vectorize the training text data using Tokenizer and pad_sequences
max_length = 200
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=max_length)

y_train_categorical = to_categorical(y_train, num_classes=2)

# Display the shapes of the resulting matrices
print("Shape of X_train:", X_train.shape)
print("Shape of y_train_categorical:", y_train_categorical.shape)

# Tokenize and vectorize the testing text data using the same Tokenizer
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=max_length)

# Display the shape of X_test
print("Shape of X_test:", X_test.shape)


Shape of X_train: (21367, 200)
Shape of y_train_categorical: (21367, 2)
Shape of X_test: (5342, 200)


In [40]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

embedding_dim = 50

# Define the vocabulary size based on the actual number of unique words in the training data
vocab_size = len(tokenizer.word_index) + 1

max_length = 200

m1 = Sequential()
m1.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
m1.add(LSTM(units=150, dropout=0.3, recurrent_dropout=0.3))
m1.add(Dense(units=1, activation='sigmoid'))  # For binary classification
# If you have multiclass classification with three classes, change 'units' to 3 and use 'softmax' activation
# m1.add(Dense(units=3, activation='softmax'))

m1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
m1.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 200, 50)           1326800   
                                                                 
 lstm_3 (LSTM)               (None, 150)               120600    
                                                                 
 dense_3 (Dense)             (None, 1)                 151       
                                                                 
Total params: 1447551 (5.52 MB)
Trainable params: 1447551 (5.52 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [41]:
# Train the model
m1.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = m1.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy * 100:.2f}%')

Epoch 1/10
334/334 [==============================] - 67s 196ms/step - loss: 0.4986 - accuracy: 0.7555 - val_loss: 0.3614 - val_accuracy: 0.8371
Epoch 2/10
334/334 [==============================] - 80s 240ms/step - loss: 0.2351 - accuracy: 0.9070 - val_loss: 0.3217 - val_accuracy: 0.8639
Epoch 3/10
334/334 [==============================] - 80s 239ms/step - loss: 0.1279 - accuracy: 0.9537 - val_loss: 0.3785 - val_accuracy: 0.8583
Epoch 4/10
334/334 [==============================] - 79s 237ms/step - loss: 0.0697 - accuracy: 0.9766 - val_loss: 0.4870 - val_accuracy: 0.8431
Epoch 5/10
334/334 [==============================] - 79s 235ms/step - loss: 0.0404 - accuracy: 0.9863 - val_loss: 0.5510 - val_accuracy: 0.8478
Epoch 6/10
334/334 [==============================] - 79s 236ms/step - loss: 0.0260 - accuracy: 0.9910 - val_loss: 0.6479 - val_accuracy: 0.8491
Epoch 7/10
334/334 [==============================] - 79s 237ms/step - loss: 0.0184 - accuracy: 0.9941 - val_loss: 0.6573 - val_ac

In [42]:
from sklearn.metrics import precision_score, recall_score

# Predict on validation data
y_val_pred_prob_m1 = m1.predict(X_test)
y_val_pred_m1 = (y_val_pred_prob_m1 > 0.5).astype(int)  # Threshold for binary classification

# Assuming y_test is in binary format (0 or 1)
y_val_true_m1 = y_test

# Calculate precision and recall for binary classification
precision_m1 = precision_score(y_val_true_m1, y_val_pred_m1)
recall_m1 = recall_score(y_val_true_m1, y_val_pred_m1)

# print the results
print(f'Precision: {precision_m1:.4f}')
print(f'Recall: {recall_m1:.4f}')

167/167 [==============================] - 4s 25ms/step
Precision: 0.8401
Recall: 0.7997


In [43]:
from keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to preprocess user input and make a prediction
def predict_sarcasm(user_input, model, tokenizer, max_length):
    # Tokenize and preprocess user input
    user_input_sequence = tokenizer.texts_to_sequences([user_input])
    user_input_padded = pad_sequences(user_input_sequence, maxlen=max_length)

    # Make prediction
    prediction_prob = model.predict(user_input_padded)
    predicted_label = np.argmax(prediction_prob)

    return predicted_label, prediction_prob

# Example usage:
user_input = input("Enter a sentence: ")

# Call the predict_sarcasm function
predicted_label, predicted_prob = predict_sarcasm(user_input, m1, tokenizer, max_length)

# Display the prediction
if predicted_label == 1:
    print("The model predicts that the input is sarcastic.")
else:
    print("The model predicts that the input is not sarcastic.")

# Optional: Display the predicted probabilities
print("Predicted Probabilities:", predicted_prob)


1/1 [==============================] - 0s 30ms/step
The model predicts that the input is not sarcastic.
Predicted Probabilities: [[0.9718941]]
User input  Best Actor Award For Pretending It Isn’t Committing Genocide – Israel
